In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib as matplot 
%matplotlib inline
from sklearn.model_selection import train_test_split 

In [4]:
df = pd.read_csv('C:/Users/Windows10/Desktop/大二.5下/Python ML Project/项目数据/RGZNXLY/RGZNXLY/ch12/HR_comma_sep.csv')
print(df.isnull().any(), "\n\n")  # 检测数据是否有缺失

satisfaction_level       False
last_evaluation          False
number_project           False
average_montly_hours     False
time_spend_company       False
Work_accident            False
left                     False
promotion_last_5years    False
sales                    False
salary                   False
dtype: bool 




In [8]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [9]:
# 列重命名
df = df.rename(columns={'satisfaction_level': "satisfaction", 'last_evaluation': 'evaluation', 'number_project': 'projectCount', 'average_montly_hours': 'averageMonthlyHours', 'time_spend_company': 'yearaAtCompany', 'Work_accident': 'workAccident', 'promotion_last_5years': 'promotion', 'sales': 'department', 'left': 'turnover'})

In [12]:
front = df['turnover']
df.drop(labels=['turnover'], axis=1, inplace=True) #删除原始列
df.insert(0, 'turnover', front) #加入front列， 0为列号
df.head()

,turnover,satisfaction,evaluation,projectCount,averageMonthlyHours,yearaAtCompany,workAccident,promotion,department,salary
0,1,0.38,0.53,2,157,3,0,0,sales,low
1,1,0.80,0.86,5,262,6,0,0,sales,medium
2,1,0.11,0.88,7,272,4,0,0,sales,medium
3,1,0.72,0.87,5,223,5,0,0,sales,low
4,1,0.37,0.52,2,159,3,0,0,sales,low


In [16]:
# 验证正负样本平衡度
turnover_rate = df.turnover.value_counts()/len(df) # value_counts返回0和1数量
print("Demission rate: %.3f"% turnover_rate[0])

Demission rate: 0.800


In [17]:
df.describe()

,turnover,satisfaction,evaluation,projectCount,averageMonthlyHours,yearaAtCompany,workAccident,promotion
count,12504.000000,12504.000000,12504.000000,12504.000000,12504.000000,12504.000000,12504.000000,12504.000000
mean,0.200256,0.621834,0.716446,3.803503,200.721769,3.385717,0.149472,0.016555
std,0.400208,0.245010,0.169745,1.196592,49.341169,1.321437,0.356568,0.127601
min,0.000000,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000
25%,0.000000,0.450000,0.560000,3.000000,157.000000,3.000000,0.000000,0.000000
50%,0.000000,0.650000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000
75%,0.000000,0.820000,0.870000,5.000000,244.000000,4.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000


In [18]:
# cat.codes对字符型进行整型的编码
df["department"] = df["department"].astype('category').cat.codes
df["salary"] = df["salary"].astype('category').cat.codes

In [20]:
df.head()

,turnover,satisfaction,evaluation,projectCount,averageMonthlyHours,yearaAtCompany,workAccident,promotion,department,salary
0,1,0.38,0.53,2,157,3,0,0,7,1
1,1,0.80,0.86,5,262,6,0,0,7,2
2,1,0.11,0.88,7,272,4,0,0,7,2
3,1,0.72,0.87,5,223,5,0,0,7,1
4,1,0.37,0.52,2,159,3,0,0,7,1


In [21]:
target_name = 'turnover'
X = df.drop('turnover', axis = 1)
y = df[target_name]
# stratify分层采样列
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

In [22]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [24]:
# 训练决策树,生成classification_report
dtree = DecisionTreeClassifier(criterion='entropy', max_depth=3, min_weight_fraction_leaf=0.01)
dtree = dtree.fit(X_train, y_train)
print(classification_report(y_test, dtree.predict(X_test)))

precision    recall  f1-score   support

           0       0.98      0.96      0.97      1500
           1       0.86      0.92      0.89       376

   micro avg       0.95      0.95      0.95      1876
   macro avg       0.92      0.94      0.93      1876
weighted avg       0.96      0.95      0.96      1876



In [27]:
# 随机森林
rf = RandomForestClassifier(criterion='entropy', n_estimators=1000, max_depth=None, min_samples_split=10)
rf.fit(X_train, y_train)
print(classification_report(y_test, rf.predict(X_test)))

precision    recall  f1-score   support

           0       0.98      1.00      0.99      1500
           1       0.99      0.92      0.96       376

   micro avg       0.98      0.98      0.98      1876
   macro avg       0.99      0.96      0.97      1876
weighted avg       0.98      0.98      0.98      1876

